# Imports

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats

import seaborn as sns
sns.set_style('darkgrid', {'axes.facecolor': '0.9', "grid.color": ".6", "grid.linestyle": ":"})
sns.set_context("talk")

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

# Load data

In [ ]:
post_df = pd.read_csv("../data/post/POST_DATA.csv",header=1)
post_df

# Rename columns

In [ ]:
post_df.columns

In [ ]:
post_df.columns = [
    'Start Date', 
    'End Date', 
    'Response Type', 
    'IP Address', 
    'Progress',
    'Duration (seconds)', 
    'Finished', 
    'Recorded Date', 
    'Response ID',
    'Recipient Last Name', 
    'Recipient First Name', 
    'Recipient Email',
    'External Data Reference', 
    'Location Latitude', 
    'Location Longitude',
    'Distribution Channel', 
    'User Language',
    'EnterChosenID',
    'EnterRandomID',
    'What is your primary language?',
    'What is your primary language? - Other',
    'Are you a:',
    'Are you a: - Other',
    'Which of the following best describes your Race/Ethnicity?',
    'Do you identify as:',
    'Which of the following best describes the area you live in?',
    'Before today, I have received training on how to provide direct oral health care for individuals with special health care needs',
    'Before today, I have received training on how to provide direct oral health care for individuals with special health care needs - Yes. If you answered YES to a previous training, please describe here:',
    'Your program site',
    'Fluoridated products, such as fluoridated toothpaste and fluoridated water, can help improve the oral health of residents.',
    'It is normal for healthy gums to bleed when brushing teeth.',
    'Dry mouth can have a negative effect on overall oral health.',
    'Snacking throughout the day can have a negative impact on oral health.',
    'I believe I have previously received adequate training to help provide the best oral care possible to residents under my care.',
    'I believe residents under my care have oral health care needs which require further training to adequately understand and help manage.',
    'I believe I have effective techniques which I use to brush the teeth of residents under my care.',
    'I feel comfortable assisting residents in the safe use of fluoridated dental products (such as fluoridated toothpaste).',
    'I am able to confidently recognize non-verbal signs of pain in residents under my care.',
    'I would be interested in receiving additional training to help maintain the oral health of residents under my care.',
    'I feel confident that I have the knowledge to identify when residents under my care experience oral pain.',
    'Approximately, what percentage of residents under your care require assistance brushing or flossing their teeth?',
    'Approximately, what percentage of residents under your care experience bleeding when brushing their teeth?',
    'Approximately, what percentage of residents under your care experience bleeding when flossing their teeth?',
    'Approximately, what percentage of residents under your care express that they experience pain when brushing their teeth?',
    'Approximately, what percentage of residents under your care experience pain when flossing their teeth?',
    'Approximately, what percentage of residents under your care express that they experience dental pain throughout the day when they are not brushing or flossing their teeth?',
    'Approximately, how often do residents under your care go to the dentist?',
    'Approximately, how often do residents under your care go to the dentist? - Other',
    'On average, how often do most residents under your care brush their teeth?',
    'On average, how often do most residents under your care floss their teeth?',
    'How often should residents brush their teeth each day?',
    'How often should residents floss their teeth each day?',
    'Approximately, how often do residents under your care have snacks throughout the day between brushing and flossing their teeth?',
    'What is the biggest obstacle to providing excellent oral care to residents?',
    'What is the biggest obstacle to providing excellent oral care to residents? - Other',
    'What is your primary source of dental-related information?',
    'What is your primary source of dental-related information? - Other',
    'What is the biggest obstacle to receiving proper oral health care training in your facility?',
    'What is the biggest obstacle to receiving proper oral health care training in your facility? - Other',
    'Which resource would be most useful to help improve your confidence in delivering excellent oral homecare to residents under your care?',
    'Which resource would be most useful to help improve your confidence in delivering excellent oral homecare to residents under your care? - Other',
    'I am satisfied with the Smiles United training videos.',
    'I have increased my advocacy following the Smiles United training.',
    'I am better able to say what I want.',
    'I am serving in leadership or advocacy positions (ex. Coalitions, policy boards, advisory boards)',
    'RandomID']

post_df.columns

# Check `Response Type`
(There should be only "IP Address")

In [ ]:
post_df['Response Type'].value_counts()

## Check the 10 "Spam" responses

In [ ]:
post_df[(post_df['Response Type']=='Spam')]

### NOTES: On "Spam" responses
All 10 responses are incomplete and will be dropped 

# Drop unnecessary columns

In [ ]:
# drop unnecessary columns
keep = post_df.drop([
    'Start Date',
    'End Date', 
    'Response Type', 
    'IP Address', 
    'Progress', 
    #'Duration (seconds)', 
    #'Finished', 
    'Recorded Date',
    'Response ID',
    'Recipient Last Name',
    'Recipient First Name', 
    'Recipient Email', 
    'External Data Reference',
    'Distribution Channel', 
    'User Language'], axis=1)

keep.head()

In [ ]:
post_respondents = keep[1:].reset_index(drop=True)
post_respondents.head()

In [ ]:
post_respondents.shape # this is the total number of responses initiated

## Drop incomplete responses

In [ ]:
# filter out incomplete responses
post_completed = post_respondents[post_respondents['Finished']=='True'].reset_index(drop=True)
post_completed

In [ ]:
post_completed.shape # this is the total number of completed responses 

# View distribution of values in each question

In [ ]:
for column in post_completed.drop(['Location Latitude', 
                                   'Location Longitude',
                                   'Duration (seconds)',
                                   'Finished'], axis=1).columns:
    print("-"*60)
    print(f"COLUMN: '{column}'")
    print(f"UNIQUE VALUES: {len(post_completed[column].unique())}")
    print("- "*30)
    print(post_completed[column].value_counts())
    print("-"*60)

In [ ]:
post_completed[post_completed['EnterRandomID'] == '90911970'].T

## Check `Are you a:` responses
Per the exclusion criteria: "DSPs who do not provide direct care to individuals with IDD," I will delete any respondents who do not fit this criteria. 

In [ ]:
post_completed['Are you a:'].value_counts()

In [ ]:
post_completed[post_completed['Are you a:'] == "Person with a disability"].T[6:]

# Minutes to complete survey

In [ ]:
post_completed['Duration (seconds)'] = post_completed['Duration (seconds)'].astype(int)

post_time = post_completed[['Duration (seconds)']]
post_time['mins'] = post_time.loc[:,'Duration (seconds)']/60
post_time

In [ ]:
# pickle time df
pd.to_pickle(post_time, "../saved_data_frames/POST_time_df.pkl")

# Set up visualization of completed surveys

In [ ]:
print(f'Total: {len(post_respondents)}')
print(f'Completed: {len(post_completed)}')

In [ ]:
post_totals = pd.DataFrame([len(post_respondents),len(post_completed)], 
                           columns=['count'], index=['Total', 'Completed'])
post_totals

In [ ]:
# pickle post_totals df
pd.to_pickle(post_totals, "../saved_data_frames/POST_totals_df.pkl")

## Set up Lat Long df

In [ ]:
d = post_completed[['Location Latitude','Location Longitude']]

In [ ]:
d = d[['Location Latitude','Location Longitude']]
d['lat,long'] = list(zip(d['Location Latitude'], d['Location Longitude']))

lat_long_POST = pd.DataFrame(d['lat,long'].value_counts()).reset_index()
lat_long_POST.columns = ['(lat,long)','count']
pd.to_pickle(lat_long_POST, "../saved_data_frames/lat_long_POST_df.pkl")

In [ ]:
lat_long_POST

## Drop remaining unnessesary columns:
Drop:
- `Finished`
- `Duration (seconds)`
- `Location Latitude`
- `Location Longitude`
- `RandomID`

In [ ]:
post_completed.drop(['Finished', 
                     'Duration (seconds)', 
                     'Location Latitude', 
                     'Location Longitude', 
                     'RandomID'], 
                    axis=1, 
                    inplace=True)
#post_completed

# Clean each questions responses

## Questions that need to be consolidated:

In [ ]:
# extract out the questions with non null responses and their paired OTHER/YES questions

for question in enumerate(post_completed.columns):

    if ("Other" in question[1]) or ("Yes" in question[1]): # if an OTHER or YES question
        i = question[0]
        col = post_completed.columns[i]
        
        if post_completed[col].value_counts().sum() > 0: # if any NON NULL responses
            prior_q = i-1
            print(f"({prior_q}, '{post_completed.columns[prior_q]}')")
            print(question)
            print()


In [ ]:
def view_values(col_num, df=post_completed):
    return df[df.columns[col_num]].value_counts()

In [ ]:
view_values(9)

In [ ]:
view_values(10)

In [ ]:
def consolidate(df, col_num1, col_num2, Other_string, convert_dict):
    col1 = df.columns[col_num1]
    col2 = df.columns[col_num2]
    
    # fill null values
    df[col2] = df[col2].fillna("Not specified")
    
    # empty list to hold new column values
    new_col_values = []
    
    # consolidate into the list
    for i in df.index:
        response = df[col1][i]
        if response == Other_string:
            new_col_values.append(df[col2][i])
        else: 
            new_col_values.append(response)
        
    # add new column
    new_col_name = col1+"_consolidated"
    #new_row = df.columns[-1] + 1
    df[new_col_name]= new_col_values
    
    # convert with dict
    df[new_col_name] = df[new_col_name].replace(convert_dict)
    
    # drop original rows
    #df = df.drop([col1, col2], axis=1)
    
    return df

In [ ]:
nine_ten_convert_dict = {
    "Not specified": "Yes - Not specified",
    "Yes": "Yes - Not specified",
    "relias": "Yes - Unknown",
    "YES,Iwas trained by the nurse": "Yes - Current or previous employment",
    "dental hygienist": "Yes - Dental professionals",
    "Taking care and brushing of individual teeth ": "Yes - Current or previous employment",
    "Trainings at work.": "Yes - Current or previous employment",
    "Anderson Center for Autism ": "Yes - Current or previous employment",
    "Smiles Post Training ": "Yes - Smiles United Training",
    "Another staff assisted" : "Yes - Current or previous employment"
    }

In [ ]:
post_completed = consolidate(
    post_completed, 9, 10, 
    "Yes. If you answered YES to a previous training, please describe here:", 
    nine_ten_convert_dict
)


In [ ]:
view_values(len(post_completed.columns)-1)

In [ ]:
view_values(29)

In [ ]:
view_values(30)

In [ ]:
twntynine_thirty_convert_dict = {
    "i dont work in the residents": "Unknown",
    "i have seen any appointment so far": "Unknown",
    "varies" : "Varies",
    "As needed": "Varies",
    "i have not seen yet": "Unknown", 
    "never": "Never"
}

In [ ]:
post_completed = consolidate(
    post_completed, 29, 30, 
    "Other (please describe):", 
    twntynine_thirty_convert_dict
)


In [ ]:
view_values(len(post_completed.columns)-1)

In [ ]:
view_values(36)

In [ ]:
view_values(37)

In [ ]:
thrtysix_thrtyseven_convert_dict = {
    "none": "Not Specifed",
    "all of the above": "All of the above",
    "nothing" : "Not Specifed" 
}

In [ ]:
post_completed = consolidate(
    post_completed, 36, 37, 
    "Other (please describe):", 
    thrtysix_thrtyseven_convert_dict
)

In [ ]:
view_values(len(post_completed.columns)-1)

In [ ]:
view_values(38)

In [ ]:
view_values(39)

In [ ]:
thrtyeight_thrtynine_convert_dict = {
    "above all": "All of the above",
    "All above": "All of the above" 
}

In [ ]:
post_completed = consolidate(
    post_completed, 38, 39, 
    "Other (please describe):", 
    thrtyeight_thrtynine_convert_dict
)

In [ ]:
view_values(len(post_completed.columns)-1)

In [ ]:
view_values(40)

In [ ]:
view_values(41)

In [ ]:
forty_fortyone_convert_dict = {
    "havent fac nothing like that.": "Unknown",
    "staffing, behaviors, knowledge": "Staffing, Behaviors, Knowledge",
    "all of the above" : "All of the above",
    "na": "Not Specified",
    "none": "There are no obstacles to receiving proper oral health care training in my facility"
}

In [ ]:
post_completed = consolidate(
    post_completed, 40, 41, 
    "Other (please describe):", 
    forty_fortyone_convert_dict
)

In [ ]:
view_values(len(post_completed.columns)-1)

In [ ]:
view_values(42)

In [ ]:
view_values(43)

In [ ]:
post_completed = consolidate(
    post_completed, 42, 43, 
    "Other (please explain):", 
    {"all of the above": "All of the above"}
)

In [ ]:
view_values(len(post_completed.columns)-1)

In [ ]:
# drop the original consolidated columns
post_completed = post_completed.drop(
    [
    post_completed.columns[9],
    post_completed.columns[10],
    post_completed.columns[29],
    post_completed.columns[30],
    post_completed.columns[36],
    post_completed.columns[37],
    post_completed.columns[38],
    post_completed.columns[39],
    post_completed.columns[40],
    post_completed.columns[41],
    post_completed.columns[42],
    post_completed.columns[43]
                    ], 
    axis=1)

In [ ]:
post_completed

In [ ]:
need_to_drop = []
for col in post_completed.columns:
    if len(post_completed[col].value_counts()) == 0:
        need_to_drop.append(col)
        
post_completed = post_completed.drop(need_to_drop, axis=1)

In [ ]:
#replace remaining nulls
post_completed = post_completed.fillna("Not Specified") 
# replace none string with just "None"
post_completed = post_completed.replace(
    to_replace="None of the residents under my care experience bleeding when brushing their teeth",
    value="None"
)

post_completed.info()

In [ ]:
for column in post_completed.columns:
    print("-"*60)
    print(f"COLUMN: '{column}'")
    print(f"UNIQUE VALUES: {len(post_completed[column].unique())}")
    print("- "*30)
    print(post_completed[column].value_counts())
    print("-"*60)

In [ ]:
# bam the data frame has been cleaned. Now I need to save it. 
pd.to_pickle(post_completed, "../saved_data_frames/cleaned_post.pkl")

In [ ]:
# use this find the exact response (shows hidden spaces)
for response in post_completed[post_completed.columns[-1]]:
    if "Smiles" in response:
        print(f'"{response}"')